In [1]:
import sys
sys.path.append("..")

# backends
from qiskit.providers.fake_provider import Fake27QPulseV1,Fake5QV1,Fake20QV1
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2


# poly_sabre
from poly_sabre.poly_circuit_preprocess import *
from poly_sabre.poly_circuit_utils import *
from poly_sabre.poly_sabre import POLY_SABRE


# qiskit
from qiskit import QuantumCircuit, transpile


import cProfile
import pstats

In [2]:
file_path = "../benchmarks/ibmqx/4gt5_76.json"
data = json_file_to_isl(file_path)
edges = FakeGuadalupeV2().configuration().coupling_map

In [3]:
poly_sabre = POLY_SABRE(edges,data)

In [4]:
with cProfile.Profile() as pr:
    initial_mapping = poly_sabre.get_initial_mapping()
    swaps = poly_sabre.run(initial_mapping, num_iter=1)
    print(swaps)

27


In [5]:
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME) 
stats.print_stats(10)

         30947 function calls (30760 primitive calls) in 2.091 seconds

   Ordered by: internal time
   List reduced from 512 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4152    0.824    0.000    0.839    0.000 /home/merwan/Documents/quantum/sabre/new/../poly_sabre/poly_heuristic.py:79(calculate_distance)
        2    0.598    0.299    1.958    0.979 /home/merwan/Documents/quantum/sabre/new/../poly_sabre/poly_sabre.py:36(execute_sabre_algorithm)
       24    0.260    0.011    0.265    0.011 /home/merwan/Documents/quantum/sabre/new/../poly_sabre/poly_sabre.py:110(find_best_node)
      312    0.114    0.000    0.114    0.000 /home/merwan/Documents/quantum/sabre/new/../poly_sabre/poly_sabre.py:151(update_mapping)
        2    0.108    0.054    0.108    0.054 /home/merwan/Documents/quantum/sabre/new/../poly_sabre/poly_circuit_preprocess.py:115(get_dag)
      288    0.069    0.000    0.950    0.003 /home/merwan/Documents/quantu

In [6]:
circuit = QuantumCircuit.from_qasm_str(data["qasm_code"])
transpiled_circuit = transpile(circuit, coupling_map=edges, optimization_level=3)
sabre_swap_count = transpiled_circuit.count_ops().get('swap') 
print(sabre_swap_count)

24
